# Setup and visualization of a digital elevation model

**Important:** This example requires additional data which can be downloaded with the following command-line call: `python download.py algeria`.

In this example, we demonstrate how to set up minimal infrastructure to add a triangulated digital elevation model (DEM) to an Eradiate simulation. We use the expert interface to create RGB images of the Algeria-5 pseudo-invariant calibration site.

This notebook assumes that the data required for the simulations is available at the root of the repository, in a `data` subdirectory.

In [ ]:
# Let's start with a few imports.
import eradiate

# In addition to Eradiate, we need:
# - matplotlib to plot our results
import matplotlib.pyplot as plt

# - Numpy and xarray for vectorized array calculus
import numpy as np

# - Mitsuba (Eradiate's radiometric kernel) to use the expert interface
import mitsuba as mi

# We also import a few Eradiate types that will be used in the following:
from eradiate import KernelContext, fresolver
from eradiate import unit_registry as ureg
from eradiate.kernel import (
    KernelSceneParameterFlags,
    SceneParameter,
    SearchSceneParameter,
)
from eradiate.scenes.spectra import InterpolatedSpectrum

In [ ]:
# Now, let's do a bit of preparation. We first select an Eradiate mode
# (that should be done as early as possible in the computational workflow):
eradiate.set_mode("mono")

# Next, we add the data directory to the file resolver.
fresolver.append("../data")

# Finally, we adjust the plotting style (optional)
eradiate.plot.set_style()
plt.style.use("eradiate.mplstyle")

Now that the basic environment setup is complete, let's briefly introduce the scene build method.

* We will use a **spherical-shell geometry** for the atmosphere, and we assume a spherical geoid surface.
* Eradiate centers the planetary surface (a sphere of radius *r* = 6378.1 km) at the origin (0, 0, 0) and positions sensors by default at the North Pole.
* The DEM has already been triangulated as a mesh stored in the PLY format. The mesh was extracted from the Copernicus GLO-30 dataset, and vertices were already moved to position the reference at the North Pole.

In [ ]:
# We retrieve the radius of the sphere used as the geoid. Mesh vertex positions
# are given in metres, so we won't change the default kernel units, and we will
# program everything in metres.
earth_radius = eradiate.constants.EARTH_RADIUS.m_as("m")

# When creating the mesh, the elevation data in the DEM dataset was applied
# relative to the geoid. Therefore, there is an offset between the geoid level
# and the actual surface. We define the location of the reference point in the
# scene, accounting for the elevation of the site, in local East-North-Up (ENU)
# coordinates. The center of the Algeria-5 site is approximately located at
# 31.02N 2.23E, with an elevation of 530 m.
# We also store the location ENU frame vectors for convenience:
site_center = [0, 0, earth_radius + 530.0] * ureg.m
site_center_north = np.array([0, 1, 0])
site_center_east = np.array([1, 0, 0])
site_center_up = np.array([0, 0, 1])

**Note:** For more advanced positioning, the [pymap3d](https://github.com/geospace-code/pymap3d/) library provides a convenience interface to transform geodetic coordinates into ECEF, ENU or other coordinate systems.

Now, let's define the various objects that will reflect light in our scene. We do this through Eradiate's expert interface, using Mitsuba primitives directly. We store these definitions in a *kernel dictionary*:

In [ ]:
fname_ply = fresolver.resolve(
    # "ply/algeria_5.ply"  # Full resolution (30 m)
    "ply/algeria_5x3.ply"  # Downsampled 3 times (90 m), default for best speed
)
fname_ply = str(
    fname_ply
)  # Cast to str (Mitsuba does not accept pathlib.Path instances)

kdict = {
    # First, materials. We have only one (a diffuse BRDF for sand, which
    # we will apply to both the triangulated DEM and the background planetary
    # surface). We have to be careful with the identification of the various
    # entries in the scene tree: Mitsuba parses the scene dictionary in the
    # alphanumerical order of the dictionary keys, and we want materials to be
    # processed first.
    "01_mat_sand": {
        "type": "diffuse",
        "id": "01_mat_sand",  # For safety, make this consistent with the dict key
    },
    # Next, shapes. We have the DEM first, then the background sphere.
    "02_shape_algeria-5": {
        "type": "ply",
        "filename": fname_ply,
        "face_normals": True,  # Very important: do not interpolate surface normals to avoid visual artefacts
        "bsdf": {
            "type": "ref",
            "id": "01_mat_sand",
        },  # Reference the previously defined material
    },
    "02_shape_background": {
        "type": "sphere",
        "radius": earth_radius,  # Mitsuba does not support units the way Eradiate does: this is why we needed to convert EARTH_RADIUS to metres
        "bsdf": {
            "type": "ref",
            "id": "01_mat_sand",
        },  # Reference the previously defined material
    },
}

If we'd perform a radiative transfer simulation on this only, the surface would appear white: we need to input the spectral variations of the sand BRDF's reflectance. Let's first start by loading some spectral data, which we will extract from an RPV parameter dataset:

In [ ]:
# Load the reflectance spectrum data
reflectance = fresolver.load_dataset("bsdf/rpv_sand.nc")["rho_0"]

# Initialize an InterpolatedSpectrum object to get a convenient interface to
# evaluate the spectrum (we will used its eval() method right after that)
sand_spectrum = InterpolatedSpectrum.from_dataarray(dataarray=reflectance)


# Encapsulate the call to InterpolatedSpectrum.eval() in function that can be
# used directly to declare a kernel parameter in the expert interface
def sand_reflectance(ctx: KernelContext) -> float:
    return sand_spectrum.eval(ctx.si).m_as("dimensionless")

We must let Eradiate know that at every iteration of the spectral loop, it has to update the kernel object that holds the reflectance of the sand using these data, based on the spectral context of the current loop iteration. We define this *update protocol* using the `SceneParameter` class. In that case, it should encapsulate a callable with signature `f(ctx: KernelContext) -> float`, which returns the value of the reflectance for the given kernel context (which, among others, holds information about the current spectral loop iteration).

In addition, it contains metadata that helps Mitsuba find which kernel-level parameter is associated to the callable. This is required because the way Mitsuba exposes scene parameters is currently fragile and might be unreliable if those hints are missing.

In [ ]:
kpmap = {
    # This key is arbitrary. For clarity, however, we set it consistent with
    # the expected parameter path
    "01_mat_sand.reflectance.value": SceneParameter(
        sand_reflectance,
        search=SearchSceneParameter(
            node_type=mi.BSDF,  # The type of the node in the Mitsuba scene tree that is expected to hold the parameter
            node_id="01_mat_sand",  # The ID of the node in the Mitsuba scene tree that is expected to hold the parameter
            parameter_relpath="reflectance.value",  # The parameter ID, relative to the node in the Mitsuba scene tree that is expected to hold the parameter
        ),
        flags=KernelSceneParameterFlags.SPECTRAL,  # This flag tells Eradiate that this parameter has to be updated at each iteration of the spectral loop
    )
}

Debugging ill-defined parameters can be difficult without basic knowledge of Mitsuba. For an introduction, see the [Mitsuba documentation](https://mitsuba.readthedocs.io/).

With that done, we can now move on to setting up our illumination and sensor.

In [ ]:
# These illumination parameters are arbitrary. You can use a library like skyfield
# to get the local ephemeris and set them to more realistic values.
illumination = {"type": "directional", "zenith": 30.0, "azimuth": 200.0}

# We position a camera looking at the target center, from 1 km East, 50 m above
# the reference point's altitude.
t = site_center.m
o = t + site_center_east * 1000.0 + site_center_up * 50.0
u = site_center_up
measure = {
    "type": "perspective",
    # "film_resolution": (160, 80),  # Fast preview
    "film_resolution": (750, 375),  # Medium resolution
    # "film_resolution": (1500, 750),  # High resolution
    "far_clip": 1e20,  # Set it to a very large value to prevent any clipping
    "target": t,
    "origin": o,
    "up": u,
    "srf": {
        "type": "delta",
        "wavelengths": [440, 550, 660],  # We process 3 channels to create an RGB image
    },
}

# We define a basic atmosphere with a thin aerosol layer
atmosphere = {
    "type": "heterogeneous",
    "molecular_atmosphere": {"type": "molecular"},
    "particle_layers": {
        "type": "particle_layer",
        "bottom": 0.0,
        "top": 10e3,
        "distribution": {"type": "exponential"},
        "tau_ref": 0.1,
        "dataset": "govaerts_2021-desert",
    },
}

Now, we can create an `AtmosphereExperiment` that will incorporate the Mitsuba primitives created with the expert interface, the background 1D, spherical-shell atmosphere, the illumination and measure we defined:

In [ ]:
exp = eradiate.experiments.AtmosphereExperiment(
    geometry="spherical_shell",
    surface=None,  # Important: remove the automatic surface (we took care of it manually)
    atmosphere=atmosphere,
    illumination=illumination,
    measures=measure,
    kdict=kdict,
    kpmap=kpmap,
)

# We immediately initialize the simulation. This will load the kernel scene and
# initialize internal data structures required for efficient ray tracing. Larger
# meshes will take more time to initialize.
exp.init()

We can run the simulation. The default sample count is low so it finishes quickly on most machines.

In [ ]:
spp = 16  # Quick preview
# spp = 1024  # Nice render with low noise
result = eradiate.run(exp, spp=spp)

Now that the computation is finished, we can use the `dataarray_to_rgb()` helper to visualize the RGB image we just created:

In [ ]:
img = (
    eradiate.xarray.interp.dataarray_to_rgb(
        result.squeeze()["radiance"],
        channels=[("w", 660), ("w", 550), ("w", 440)],
        gamma_correction=True,
    )
    * 1.8  # Adjust brightness with this multiplier
)
plt.imshow(img)
plt.axis("off")

Tips:

* To iterate faster when building the scene, remove as much detail as possible (*e.g.* remove the atmosphere and DEM to check it your material is correctly configured; use a coarse version of the surface mesh if available).

What to do next:

* Move the camera to a high altitude position (*e.g.* around 100 km), point it down and adjust its field of view to get a nadir view of the site.
* Load the native-resolution DEM triangulation and compare the recorded reflected radiance with the low-resolution one.
* Remove the `kpmap` from the simulation: the surface should now appear white because there is no longer a way for Eradiate to know how to update the scene reflectance at each iteration of the spectral loop.
* Replace the diffuse BRDF with the RPV BRDF.
* Apply the reflectance computation techniques discussed in the dedicated example to this 3D site.